In [ ]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import webbrowser
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut



vehicles_df = pd.read_csv('vehicles_df.csv')

# Aggregate data by city
city_counts = vehicles_df.groupby('City').size().reset_index(name='Count')

# Geocode cities to get latitude and longitude
geolocator = Nominatim(user_agent="geoapi")
city_coordinates = []

for city in city_counts['City'].unique():
    try:
        location = geolocator.geocode(f"{city}, Illinois")  # Update state/region as needed
        if location:
            city_coordinates.append({
                "City": city,
                "latitude": location.latitude,
                "longitude": location.longitude
            })
        else:
            city_coordinates.append({
                "City": city,
                "latitude": None,
                "longitude": None
            })
    except GeocoderTimedOut:
        city_coordinates.append({
            "City": city,
            "latitude": None,
            "longitude": None
        })
# Convert geocoded coordinates to DataFrame and merge with city counts
city_coordinates_df = pd.DataFrame(city_coordinates)
city_data = pd.merge(city_counts, city_coordinates_df, on="City", how="left")

# Filter out cities with missing coordinates
city_data = city_data.dropna(subset=["latitude", "longitude"])

# Initialize Dash App
app = dash.Dash(__name__)

app.layout = html.Div([
        # Title and Introduction
    html.Div([
        html.H1("Vehicle Hierarchy Dashboard", style={'textAlign': 'center'}),
        html.P(
            "This dashboard provides an interactive view of vehicle registrations, categorized by their "
            "make, model, and color. Use the dropdown below to filter by fuel type and explore the hierarchy.",
            style={'textAlign': 'center', 'fontSize': '18px', 'marginBottom': '20px'}
        ),
        html.P(
            "Hover over the segments in the sunburst chart to view detailed information about the vehicles.",
            style={'textAlign': 'center', 'fontSize': '16px', 'color': 'white'}
        )
    ], style={'padding': '20px', 'backgroundColor': '#769199'}),

        html.Div([
        html.Label("Filter by Fuel Type:", style={'fontSize': '16px', 'fontWeight': 'bold', 'color': 'black', 'marginBottom': '10px'}),
        dcc.Dropdown(
            id='fuel-filter',
            options=[{'label': ft, 'value': ft} for ft in vehicles_df['Vehicle Fuel Source'].dropna().unique()],
            value=vehicles_df['Vehicle Fuel Source'].dropna().unique()[0],
            clearable=False
        )
    ], style={'padding': '20px'}),

    # Sunburst Chart
    html.Div([
        dcc.Graph(id='sunburst-chart'),
        html.P(
            "This chart represents the hierarchy of vehicles based on their make, model, and color. "
            "The size of each segment indicates the number of vehicles within that category.",
            style={'textAlign': 'center', 'fontSize': '14px', 'marginTop': '10px'}
        )
    ]),
   
    # Line Chart
    html.Div([
        html.H3("Yearly Trends of Vehicle Registrations"),
        dcc.Graph(id='line-chart'),
    ], style={'padding': '20px'}),
])

# Callback for Sunburst, Line, and City Map
@app.callback(
    [Output('sunburst-chart', 'figure'),
     Output('line-chart', 'figure'),
     Output('city-map', 'figure')],
    [Input('fuel-filter', 'value')]
)
def update_charts(fuel_type):
    # Filter DataFrame based on the selected fuel type
    filtered_data = vehicles_df[vehicles_df['Vehicle Fuel Source'] == fuel_type].copy()
    
    # Handle missing values explicitly with .loc
    filtered_data.loc[:, 'Vehicle Make'] = filtered_data['Vehicle Make'].fillna('Unknown')
    filtered_data.loc[:, 'Vehicle Model'] = filtered_data['Vehicle Model'].fillna('Unknown')
    filtered_data.loc[:, 'Vehicle Color'] = filtered_data['Vehicle Color'].fillna('Unknown')
    
    # Add a Count column for the hierarchy visualization
    filtered_data.loc[:, 'Count'] = 1
    
    # Sunburst Chart
    sunburst = px.sunburst(
        filtered_data,
        path=['Vehicle Make', 'Vehicle Model', 'Vehicle Color'],
        values='Count',
        title="Vehicle Hierarchy by Make, Model, and Color",
        hover_data={
            'Vehicle Make': True,
            'Vehicle Model': True,
            'Vehicle Color': True,
            'Count': True
        }
    )
    sunburst.update_layout(
        margin=dict(t=50, l=25, r=25, b=25),
        hoverlabel=dict(
            bgcolor="black",
            font_size=14,
            font_family="Arial"
        )
    )

    # Line Chart
    yearly_data = filtered_data.groupby('Vehicle Model Year').size().reset_index(name='Count')
    line = px.line(
        yearly_data,
        x='Vehicle Model Year',
        y='Count',
        title="Yearly Trends of Vehicle Registrations"
    )
    line.update_layout(
        xaxis_title="Year",
        yaxis_title="Number of Registrations",
        margin=dict(t=50, l=25, r=25, b=25)
    )


# Run the App
if __name__ == '__main__':
    webbrowser.open_new('http://127.0.0.1:3000/')
    app.run_server(debug=True, host='127.0.0.1', port=3000)

